In [7]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
import nltk
from spacy import load
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nlp = load('en_core_web_sm')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
df=pd.read_csv('train.csv',encoding= 'unicode_escape')
df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [10]:
df.columns

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

In [11]:
df.dropna(subset=['text'], inplace=True)
df.shape

(27480, 10)

In [12]:
df = df[['sentiment','text']]

In [13]:
df

,sentiment,text
0,neutral,"I`d have responded, if I were going"
1,negative,Sooo SAD I will miss you here in San Diego!!!
2,negative,my boss is bullying me...
3,negative,what interview! leave me alone
4,negative,"Sons of ****, why couldn`t they put them on t..."
...,...,...
27476,negative,wish we could come see u on Denver husband l...
27477,negative,I`ve wondered about rake to. The client has ...
27478,positive,Yay good for both of you. Enjoy the break - y...
27479,positive,But it was worth it ****.


In [14]:
df.shape

(27480, 2)

In [15]:
df.sentiment.value_counts()

neutral     11117
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [17]:
import re

from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [21]:
corpus = []

for i in df['text']:
    r = re.sub('[^a-zA-Z]', '',i)
    r = r.lower()
    r = r.split()
    r = [word for word in r if word not in stopwords.words('english')]
    r = [lemmatizer.lemmatize(word) for word in r]
    r = ' '.join(r)
    corpus.append(r)

In [22]:
df['text_lemma'] = corpus

df.head()

,sentiment,text,text_lemma
0,neutral,"I`d have responded, if I were going",idhaverespondedifiweregoing
1,negative,Sooo SAD I will miss you here in San Diego!!!,sooosadiwillmissyouhereinsandiego
2,negative,my boss is bullying me...,mybossisbullyingme
3,negative,what interview! leave me alone,whatinterviewleavemealone
4,negative,"Sons of ****, why couldn`t they put them on t...",sonsofwhycouldnttheyputthemonthereleaseswealre...


In [23]:
X = df['text_lemma']
y = df['sentiment']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)
print('Training Data :', X_train.shape)
print('Testing Data : ', X_test.shape)

Training Data : (18411,)
Testing Data :  (9069,)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_train_cv.shape

(18411, 18268)

In [25]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)
X_test_cv = cv.transform(X_test)
predictions = lr.predict(X_test_cv)
predictions

array(['neutral', 'neutral', 'neutral', ..., 'neutral', 'neutral',
       'neutral'], dtype=object)

In [26]:
from sklearn.metrics import classification_report
y_test_cv = cv.transform(y_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

    negative       0.94      0.01      0.01      2548
     neutral       0.41      1.00      0.58      3692
    positive       0.95      0.02      0.04      2829

    accuracy                           0.42      9069
   macro avg       0.77      0.34      0.21      9069
weighted avg       0.73      0.42      0.25      9069

